In [1]:
import pandas as pd 
import numpy as np

## 读取数据

In [2]:
df1 = pd.concat([pd.read_excel('data/2018-2019茂名（含自媒体）.xlsx', sheet_name='游记攻略'), pd.read_excel('data/2020-2021茂名（含自媒体）.xlsx', sheet_name='游记攻略')], ignore_index=True)
df2 = pd.concat([pd.read_excel('data/2018-2019茂名（含自媒体）.xlsx', sheet_name='酒店评论'), pd.read_excel('data/2020-2021茂名（含自媒体）.xlsx', sheet_name='酒店评论')], ignore_index=True)
df3 = pd.concat([pd.read_excel('data/2018-2019茂名（含自媒体）.xlsx', sheet_name='景区评论'), pd.read_excel('data/2020-2021茂名（含自媒体）.xlsx', sheet_name='景区评论')], ignore_index=True)
df4 = pd.concat([pd.read_excel('data/2018-2019茂名（含自媒体）.xlsx', sheet_name='餐饮评论'), pd.read_excel('data/2020-2021茂名（含自媒体）.xlsx', sheet_name='餐饮评论')], ignore_index=True)
df_entity = pd.read_csv('data/所有实体.csv')

In [3]:
all_entitys = df_entity['实体'].values.tolist()
all_entitys

['脆皮热狗卷',
 '兰州拉面',
 '徐闻港口码头酒店',
 '海滨公园',
 '凉拌鹅颈肉',
 '立园',
 '泡芙',
 '黄油',
 '凤凰酒店',
 '雷祖祠',
 '油条碎',
 '天马山生态旅游景区',
 '好心公园',
 '麻斜岛',
 '贵妃广场',
 '十贤祠',
 '茂名好来登商务酒店',
 '茂名私宅印象民宿',
 '灯楼角',
 '水东湾',
 '茂名君悦商务酒店人民大道店',
 '热狗',
 '水东滨红树林景区',
 '瞭望厅',
 '关东煮',
 '卤水鹅三宝',
 '沙街云吞',
 '桂花肠',
 '文武庙',
 '茂名站南广场',
 '三元塔',
 '生煎鹅腩肉',
 '榴莲千层',
 '博贺湾',
 '浪漫海岸国际旅游度假区',
 '望海楼',
 '奶油',
 '化州白云公寓',
 '新湖公园',
 '野菠萝公园',
 '生蚝',
 '榴莲蛋糕',
 '无花果萨拉米披萨',
 '火腿',
 '烧鸭饭',
 '城市',
 '炖鹅骨头汤',
 '化州豪杰商务酒店',
 '信宜金辉煌宾馆',
 '温德姆酒店',
 '自力村',
 '红树林栈道',
 '面包',
 '蔬菜',
 '露天矿公园',
 '藏龙湾',
 '早茶',
 '百灵庙',
 '文化广场',
 '茂名鑫都宾馆',
 '游乐场',
 '尚客优连锁酒店(化州站前路店)',
 '面包体',
 '化州怡沛尚公寓',
 '叉烧饭',
 '旺同竹海',
 '贡园',
 '中国第一滩一家人公寓',
 '水东港',
 '日式蟹肉卷',
 '潘茂名纪念馆',
 '浪漫海岸旅游度假区',
 '大角湾',
 '好心湖',
 '水东湾海洋公园',
 '维也纳3好酒店(茂名电城店)',
 '红树林湿地公园',
 '索性酒店(化州火车站店)',
 '阳江海上丝绸之路博物馆',
 '寿司',
 '天马山',
 '牛肚',
 '情人桥',
 '牛肉',
 '赤坎老街',
 '牛排',
 '肉松',
 '油地公园',
 '蛇口码头',
 '茂名茂南高山九树公寓',
 '玻璃桥',
 '西湖公园',
 '摩天轮',
 '滘尾角灯塔',
 '观鹭阁',
 '温德姆至尊酒店',
 '雷州西湖',
 '米饭',
 '和牛盖饭',
 '酱汁',
 '自助菜',
 '放鸡岛海上游乐',
 '蜂蜜芥末',
 '荷花

### 数据维度

In [4]:
df1.shape, df2.shape, df3.shape, df4.shape

((294, 9), (1093, 7), (1203, 5), (6984, 6))

## 数据处理

### 去除换行符

In [5]:
df1['正文'] = df1['正文'].apply(lambda text: str(text).replace('\n', '').replace(' ', ''))
df2['评论内容'] = df2['评论内容'].apply(lambda text: str(text).replace('\n', ''))
df3['评论内容'] = df3['评论内容'].apply(lambda text: str(text).replace('\n', ''))
df4['评论内容'] = df4['评论内容'].apply(lambda text: str(text).replace('\n', ''))

In [6]:
df1.columns, df2.columns, df3.columns, df4.columns

(Index(['游记ID', '城市', '游记标题', '发布时间', '出发时间', '出行天数', '人物', '人均费用', '正文'], dtype='object'),
 Index(['酒店评论ID', '城市', '酒店名称', '评论日期', '评论内容', '入住日期', '入住房型'], dtype='object'),
 Index(['景区评论ID', '城市', '景区名称', '评论日期', '评论内容'], dtype='object'),
 Index(['餐饮评论ID', '城市', '餐饮名称', '评论日期', '评论内容', '标题'], dtype='object'))

## BIO标注数据

In [7]:
def entity_code(data=None, name=None, entity=None):
    # 标注实体
    text_code_bi = []
    for cont in data[name]:
        for s in entity:
            if s in cont:
                cont = str(cont).replace(s, 'B' + 'I'*(len(s)-1))
        text_code_bi.append(cont)
    text_code_list = []
    # 标注非实体
    for cont in text_code_bi:     
        for w in list(cont):
            if w not in 'BIO':
                cont = cont.replace(w, 'O')
        text_code_list.append(cont)
    data['实体编码'] = text_code_list
    return data

### 标注实体

In [8]:
df_1 = entity_code(df1, '正文', all_entitys)
df_2 = entity_code(df2, '评论内容', all_entitys)
df_3 = entity_code(df3, '评论内容', all_entitys)
df_4 = entity_code(df4, '评论内容', all_entitys)

In [9]:
df_1['实体编码']

0      OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1      OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
2      OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3      OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
4      OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
                             ...                        
289    OOOBIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
290    OOOOOBIIIIOOOOOOOOBIIIIOOOOOOOOOOOOOOOOOOOOOOO...
291    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
292    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
293    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
Name: 实体编码, Length: 294, dtype: object

In [10]:
df_2['实体编码']

0                                                 OOOOOOO
1                                                OOOOOOOO
2       OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3                                         OOOOOOOOOOOOOOO
4                                      OOOOOOOOOOOOOOOOOO
                              ...                        
1088                                    OOOOOOOOOOOOOOOOO
1089                                    OOOOOOOOOOOOOOOOO
1090                       OOOOOOOOOOOOOOOOOOOOOOOOOOOOBI
1091    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1092                                   OOOOOOOOOOOOOOOOOO
Name: 实体编码, Length: 1093, dtype: object

In [11]:
df_3['实体编码']

0                                           OOOOOOOOOOOOO
1       OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
2       OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3           OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
4       OOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
                              ...                        
1198    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1199    BIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1200    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1201               OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
1202    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
Name: 实体编码, Length: 1203, dtype: object

In [12]:
df_4['实体编码']

0                                       OOOOOO
1                               OOOOOOOOOOOOOO
2       OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
3                    OOOOOOOOOOOOOOOOOOOOOOOOO
4           OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
                         ...                  
6979                       OOOOOOOOOOOOOOOOOOO
6980                             OOOOOOOOOOOOO
6981                                       OOO
6982                                       OOO
6983                                    OOOOOO
Name: 实体编码, Length: 6984, dtype: object

### 保存数据

In [13]:
df_1.to_csv('data/游记攻略标注后的数据.csv', index=False)
df_2.to_csv('data/酒店评论标注后的数据.csv', index=False)
df_3.to_csv('data/景区标注后的数据.csv', index=False)
df_4.to_csv('data/餐饮标注后的数据.csv', index=False)